In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
from recirq.otoc.loschmidt import TiltedSquareLatticeLoschmidtSpec, TiltedSquareLatticeLoschmidtData
assert TiltedSquareLatticeLoschmidtData, 'register deserializer'
assert TiltedSquareLatticeLoschmidtSpec, 'register deserializer'


from recirq.quantum_runtime import load_raw_results
run_id = 'testrun-4'
raw_results = load_raw_results(run_id)

In [ ]:
raw_results.executable_results[0].runtime_info

In [ ]:
from recirq.otoc.loschmidt import process_results, flat_results_to_dataframe
results = process_results(raw_results)
df = flat_results_to_dataframe(results)

In [ ]:
df

In [ ]:
df.set_index(['run_id', 'width', 'height', 'macrocycle_depth', 'instance_i'])

In [ ]:
import inspect
def group_and_call(func, df):
    func_argnames = list(inspect.signature(func).parameters.keys())
    if 'df' not in func_argnames:
        raise ValueError("group_and_call requires the function `func` to accept a DataFrame argument named `df`")
    func_argnames.remove('df')
    
    def func_wrap(df):
        # the groupby values are in df.name
        kwargs = {argname: df.name[i] for i, argname in enumerate(func_argnames)}
        return func(df=df, **kwargs)
    
    return df.groupby(func_argnames).apply(func_wrap)

In [ ]:
def _plot2(df, width, height, n_qubits):
    gb = df.groupby(['macrocycle_depth']).agg(['mean', 'std']).reset_index()
    plt.errorbar(
        x = gb['macrocycle_depth'],
        y = gb['success_probability', 'mean'],
        yerr = gb['success_probability', 'std'],
        capsize=7,
        marker='o',
        label=f'{n_qubits}q ({width}x{height})'
    )

def _plot1(df, run_id, n_repetitions):
    group_and_call(_plot2, df)
    plt.legend(loc='best', title='width, height')
    plt.yscale('log')
    plt.xlabel('# Cycles per Phi')
    plt.ylabel('Success Probability')
    plt.tight_layout()
    plt.show()
    
    
_ = group_and_call(_plot1, df)